In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from PIL import Image
import numpy as np
import os
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
import cv2 
import random
from math import log
from google.colab.patches import cv2_imshow
from tqdm import tqdm

In [ ]:
image = '/kaggle/input/signature-verification-dataset/sign_data/test/049/03_049.png'
ext = ".png"
key = 20
img = cv2.imread(image)
cv2_imshow(img)

In [ ]:
def getImageMatrix(imageName):
    im = Image.open(imageName) 
    pix = im.load()
    color = 1
    if type(pix[0,0]) == int:
        color = 0
    image_size = im.size 
    image_matrix = []
    for width in range(int(image_size[0])):
        row = []
        for height in range(int(image_size[1])):
            row.append((pix[width, height]))
        image_matrix.append(row)
    return image_matrix, image_size[0], image_size[1], color

In [ ]:
# Function to get Image matrix for grayscale image
def getImageMatrix_gray(imageName):
    # Open the image and convert to grayscale ('L' mode gives grayscale)
    im = Image.open(imageName).convert('L')
    pix = im.load()
    image_size = im.size  # Image dimensions (width, height)
    
    # Initialize the matrix to store pixel values
    image_matrix = []
    
    # Iterate over each pixel in the image and store it in the matrix
    for width in range(int(image_size[0])):
        row = []
        for height in range(int(image_size[1])):
            # Append the grayscale intensity value (a single value)
            row.append(pix[width, height])
        image_matrix.append(row)
    
    return image_matrix, image_size[0], image_size[1]

In [ ]:
def ArnoldCatTransform(img, num):
    rows, cols, ch = img.shape
    n = rows
    img_arnold = np.zeros([rows, cols, ch])
    for x in range(0, rows):
        for y in range(0, cols):
            img_arnold[x][y] = img[(x + y) % n][(x + 2 * y) % n]
    return img_arnold

In [ ]:
def ArnoldCatEncryption(imageName, key):
    img = cv2.imread(imageName)
    for i in range(0, key):
        img = ArnoldCatTransform(img, i)
    cv2.imwrite(imageName.split('.')[0] + "_ArnoldcatEnc.png", img)
    return img

In [ ]:
def ArnoldCatDecryption(imageName, key):
    img = cv2.imread(imageName)
    rows, cols, ch = img.shape
    dimension = rows
    decrypt_it = dimension
    if (dimension % 2 == 0) and 5**int(round(log(dimension / 2, 5))) == int(dimension / 2):
        decrypt_it = 3 * dimension
    elif 5**int(round(log(dimension, 5))) == int(dimension):
        decrypt_it = 2 * dimension
    elif (dimension % 6 == 0) and 5**int(round(log(dimension / 6, 5))) == int(dimension / 6):
        decrypt_it = 2 * dimension
    else:
        decrypt_it = int(12 * dimension / 7)
    for i in range(key, decrypt_it):
        img = ArnoldCatTransform(img, i)
    cv2.imwrite(imageName.split('_')[0] + "_ArnoldcatDec.png", img)
    return img

In [ ]:
# Encrypt the image and save it to /kaggle/working/
ArnoldCatEncryptionIm = ArnoldCatEncryption(image, key)
encrypted_image_path = '/kaggle/working/' + image.split('/')[-1].split('.')[0] + "_ArnoldcatEnc.png"
print("Encrypted image saved at:", encrypted_image_path)
cv2_imshow(ArnoldCatEncryptionIm)

# Check if the file exists in /kaggle/working/
import os
if not os.path.exists(encrypted_image_path):
    print("Encrypted image not found:", encrypted_image_path)

# Decrypt the image from the correct path
ArnoldCatDecryptionIm = ArnoldCatDecryption(encrypted_image_path, key)
cv2_imshow(ArnoldCatDecryptionIm)
